In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [2]:
df = pd.read_csv('Movielist_DC.csv')

df = df.reindex(columns=['name', 'name_link', 
                         'critics_consensus', 'score_critical', 'score_audience', 'count_critical', 'count_audience', 'abstract',
                         'Rating', 'Genre', 'Directed_By', 'Written_By', 'In_Theaters', 'On_Disc_Streaming', 'Box_Office', 'Runtime', 'Studio'])\
                         .astype(str)
df#.head()

,name,name_link,critics_consensus,score_critical,score_audience,count_critical,count_audience,abstract,Rating,Genre,Directed_By,Written_By,In_Theaters,On_Disc_Streaming,Box_Office,Runtime,Studio
0,Superman and the Mole-Men,superman-and-the-molemen,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Batman: The Movie,batman_the_movie,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Wonder Woman(2009),wonder-woman,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Superman,superman_the_movie,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Superman II,superman_ii,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,Swamp Thing,swamp_thing,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,Superman III,superman_iii,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,Supergirl,supergirl,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,Superman IV: The Quest for Peace,superman_iv_the_quest_for_peace,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,The Return of Swamp Thing,return_of_swamp_thing,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [3]:
# check the set of the labels
label_set = set()
for i in range(len(df)):
    time.sleep(1)
    # fetch html by url
    url = 'https://www.rottentomatoes.com/m/' + df['name_link'][i]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    tag = soup.find_all('div',class_="meta-label subtle")
    set_temp = set(map(lambda x: x.text.strip(), tag))
    label_set = label_set.union(set_temp)
    print(i, len(set_temp), df['name_link'][i])
print(len(label_set))
label_set

(0, 6, 'superman-and-the-molemen')
(1, 8, 'batman_the_movie')
(2, 8, 'wonder-woman')
(3, 8, 'superman_the_movie')
(4, 8, 'superman_ii')
(5, 7, 'swamp_thing')
(6, 8, 'superman_iii')
(7, 8, 'supergirl')
(8, 8, 'superman_iv_the_quest_for_peace')
(9, 8, 'return_of_swamp_thing')
(10, 8, '1001781_batman')
(11, 8, 'batman_returns')
(12, 8, 'batman_mask_of_the_phantasm')
(13, 8, 'tank_girl')
(14, 8, 'batman_forever')
(15, 3, 'justice_league_of_america_1997')
(16, 8, '1077027_batman_and_robin')
(17, 8, '1078035_steel')
(18, 5, 'gen-13')
(19, 9, 'road_to_perdition')
(20, 9, 'league_of_extraordinary_gentlemen')
(21, 9, 'american_splendor')
(22, 9, 'catwoman')
(23, 9, 'constantine')
(24, 9, 'v_for_vendetta')
(25, 9, 'batman_begins')
(26, 9, 'history_of_violence')
(27, 9, 'superman_returns')
(28, 9, 'stardust')
(29, 9, 'the_dark_knight')
(30, 9, 'the_spirit')
(31, 9, 'watchmen')
(32, 9, '1226860_losers')
(33, 9, 'jonah_hex')
(34, 9, 'red')
(35, 9, 'green_lantern')
(36, 9, 'the_dark_knight_rises')
(

{u'Box Office:',
 u'Directed By:',
 u'Genre:',
 u'In Theaters:',
 u'On Disc/Streaming:',
 u'Rating:',
 u'Runtime:',
 u'Studio:',
 u'Written By:'}

In [4]:
%%time
for i in range(len(df)):
    print(i)
    time.sleep(1)
    # fetch html by url
    url = 'https://www.rottentomatoes.com/m/' + df['name_link'][i]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # evaluation
    tag = soup.find_all('p',class_="mop-ratings-wrap__text mop-ratings-wrap__text--concensus")
    df.at[i,'critics_consensus'] = tag[0].text

    tag = soup.find_all('span',class_="mop-ratings-wrap__percentage")
    df.at[i,'score_critical'] = tag[0].text.split('\n')[1].strip()

    tag = soup.find_all('span',class_="mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience")
    df.at[i,'score_audience'] = tag[0].text.split('\n')[1].strip()

    tag = soup.find_all('small',class_="mop-ratings-wrap__text--small")
    df.at[i,'count_critical'] = tag[0].text.split('\n')[2].strip()
    df.at[i,'count_audience'] = tag[1].text.split('\n')[2].strip()

    tag = soup.find_all('div',class_="movie_synopsis clamp clamp-6")
    df.at[i,'abstract'] = tag[0].text.strip()
    
    # meta value
    # labels
    tag = soup.find_all('div',class_="meta-label subtle")
    labels = map(lambda x: x.text.strip(), tag)
    
    # values
    tag = soup.find_all('div',class_="meta-value")
    try:
        index = labels.index('Rating:')
        df.at[i,'Rating'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Genre:')
        list_Genre = tag[index].text.replace(',', '').split('\n')
        list_Genre = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Genre))
        df.at[i,'Genre'] = list_Genre
    except:
        pass
        
    try:
        index = labels.index('Directed By:')
        list_Directed_By = tag[index].text.replace(',', '').split('\n')
        list_Directed_By = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Directed_By))
        df.at[i,'Directed_By'] = list_Directed_By
    except:
        pass
        
    try:
        index = labels.index('Written By:')
        list_Written_By = tag[index].text.replace(',', '').split('\n')
        list_Written_By = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Written_By))
        df.at[i,'Written_By'] = list_Written_By
    except:
        pass

    try:
        index = labels.index('In Theaters:')
        df.at[i,'In_Theaters'] = tag[index].text.strip().split('\n')[0]
    except:
        pass

    try:
        index = labels.index('On Disc/Streaming:')
        df.at[i,'On_Disc_Streaming'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Box Office:')
        df.at[i,'Box_Office'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Runtime:')
        df.at[i,'Runtime'] = tag[index].text.strip()
    except:
        pass

    try:
        index = labels.index('Studio:')
        list_Studio = tag[index].text.replace(',', '').split('\n')
        list_Studio = filter(lambda x : len(x)>0, map(lambda x: x.strip(), list_Studio))
        df.at[i,'Studio'] = list_Studio
    except:
        pass


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
CPU times: user 19.9 s, sys: 48 ms, total: 20 s
Wall time: 1min 7s


In [5]:
df.to_csv('BasicInfo_DC.csv', encoding='utf-8')
df#.head()

,name,name_link,critics_consensus,score_critical,score_audience,count_critical,count_audience,abstract,Rating,Genre,Directed_By,Written_By,In_Theaters,On_Disc_Streaming,Box_Office,Runtime,Studio
0,Superman and the Mole-Men,superman-and-the-molemen,No consensus yet.,23%,23%,N/A,571,Reporters Clark Kent and Lois Lane arrive in t...,NR,"[Action & Adventure, Kids & Family, Science Fi...",[Lee Sholem],nan,"Nov 23, 1951","May 1, 2001",nan,58 minutes,nan
1,Batman: The Movie,batman_the_movie,Batman: The Movie elevates camp to an art form...,77%,62%,31,"138,834","Tongue-in-cheek humor prevails in Batman, a wi...",PG,"[Action & Adventure, Drama, Science Fiction & ...",[Leslie H. Martinson],[Lorenzo Semple Jr.],"Jul 30, 1966","Apr 23, 2002",nan,104 minutes,[20th Century Fox]
2,Wonder Woman(2009),wonder-woman,No consensus yet.,82%,78%,11,"8,535",A graceful Amazon princess of untold strength ...,PG-13 (for violence throughout and some sugges...,"[Action & Adventure, Animation, Drama, Kids & ...",[Lauren Montgomery],[Michael Jelenic],"Mar 3, 2009","Mar 3, 2009",nan,93 minutes,[Warner Bros. Animation]
3,Superman,superman_the_movie,"Superman deftly blends humor and gravitas, tak...",94%,86%,66,"285,768",Christopher Reeve as the Man of Steel in a big...,"PG (for peril, some mild sensuality and language)","[Action & Adventure, Drama]",[Richard Donner],"[Mario Puzo, Leslie Newman, Robert Benton]","Dec 15, 1978","May 1, 2001",nan,142 minutes,[Warner Bros. Pictures]
4,Superman II,superman_ii,The humor occasionally stumbles into slapstick...,85%,75%,48,"239,798","Between giving up his super powers, confrontin...","PG (for sequences of action violence, come lan...","[Action & Adventure, Science Fiction & Fantasy]","[Richard Lester, Richard Donner]","[Mario Puzo, David Newman, Leslie Newman]","Jun 19, 1981","May 1, 2001",nan,127 minutes,[Warner Bros. Pictures]
5,Swamp Thing,swamp_thing,Unabashedly campy -- often to its detriment --...,67%,34%,36,"21,923",Historia de suspenso escrita y dirigida por We...,"PG (adult situations/language, nudity, violence)","[Comedy, Cult Movies, Horror, Science Fiction ...",[Wes Craven],nan,"Jan 1, 1982","Aug 14, 2001",nan,90 minutes,[Nelson Entertainment]
6,Superman III,superman_iii,"When not overusing sight gags, slapstick, and ...",24%,23%,46,"167,114",The Man of Steel (Christopher Reeve) takes on ...,PG,"[Action & Adventure, Kids & Family, Science Fi...",[Richard Lester],"[David Newman, Leslie Newman]","Jun 17, 1983","May 1, 2001",nan,125 minutes,[WARNER BROTHERS PICTURES]
7,Supergirl,supergirl,The effects are cheesy and Supergirl's wide-ey...,10%,26%,31,"49,201",A big-budget spin-off from the series of three...,PG,"[Action & Adventure, Science Fiction & Fantasy]",[Jeannot Szwarc],[David Odell],"Nov 21, 1984","Aug 8, 2000",nan,89 minutes,[TriStar Pictures]
8,Superman IV: The Quest for Peace,superman_iv_the_quest_for_peace,The Superman series bottoms out here: the acti...,11%,15%,44,"65,958",Superman (Christopher Reeve) tries to save the...,PG,"[Action & Adventure, Science Fiction & Fantasy]",[Sidney J. Furie],"[Lawrence Konner, Mark Rosenthal]","Jul 24, 1987","May 1, 2001",nan,91 minutes,[Warner Home Video]
9,The Return of Swamp Thing,return_of_swamp_thing,No consensus yet.,44%,28%,9,"3,694",A beautiful horticulturist visits her stepfath...,"PG-13 (adult situations/language, violence)","[Action & Adventure, Comedy, Horror, Science F...",[Jim Wynorski],"[Grant Morris, Derek Spencer, Neil Cuthbert]","May 12, 1989","Feb 18, 2003",nan,90 minutes,[Sony Pictures Home Entertainment]
